In [1]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from itertools import combinations
import time

In [2]:
# 엑셀 파일에서 당첨 번호 읽기
winner_number = pd.read_excel('winner_number.xlsx', header=None, index_col=None, engine='openpyxl')

# 당첨 번호 리스트 만들기
winner_list = winner_number.values.tolist()

print(f"총 당첨 번호 개수: {len(winner_number)}")
print(winner_number.head())  # 전체 데이터 대신 앞부분만 출력

총 당첨 번호 개수: 1139
   0   1   2   3   4   5
0  5  11  14  27  29  36
1  5  12  17  29  34  35
2  4  12  24  33  38  45
3  8  10  23  24  35  43
4  3   4  12  14  25  43


In [3]:
def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

def result_format(A):
    return f"{A}"

def calculate_similarity_matrix(winner_list):
    winner_array = np.array(winner_list, dtype=float)
    norm_winner = np.linalg.norm(winner_array, axis=1)
    similarity_matrix = np.dot(winner_array, winner_array.T) / np.outer(norm_winner, norm_winner)
    np.fill_diagonal(similarity_matrix, 0)
    return similarity_matrix

def find_most_similar_combinations(similarity_matrix, winner_list, top_n=5):
    average_similarities = similarity_matrix.mean(axis=1)
    top_indices = average_similarities.argsort()[::-1][:top_n]
    results = [(result_format(winner_list[i]), average_similarities[i]) for i in top_indices]
    return results

def get_lottery_combinations(winner_set):
    all_combinations = set(combinations(range(1, 46), 6))
    return all_combinations - winner_set

def find_top_similar(combinations, target, top_n=5):
    target_array = np.array(target, dtype=float)
    similarities = ((comb, cos_sim(np.array(comb, dtype=float), target_array)) for comb in combinations)
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]

In [4]:
# 유사도 행렬 계산
similarity_matrix = calculate_similarity_matrix(winner_list)

# 가장 유사한 조합 찾기
top_combinations = find_most_similar_combinations(similarity_matrix, winner_list)

print("가장 유사도가 높은 로또 번호 조합:")
for combo, similarity in top_combinations:
    print(f"{combo}: {similarity:.4f}")

가장 유사도가 높은 로또 번호 조합:
[7, 11, 16, 21, 27, 33]: 0.9824
[5, 13, 17, 23, 28, 36]: 0.9821
[6, 12, 19, 24, 34, 41]: 0.9821
[5, 10, 16, 24, 27, 35]: 0.9818
[7, 15, 20, 25, 33, 43]: 0.9818


In [5]:
# 메인 실행 부분
start_time = time.time()

first_combo, first_similarity = top_combinations[0]
print(f"첫 번째 조합: {first_combo}, 유사도: {first_similarity}")

winner_set = set(tuple(sorted(combo)) for combo in winner_list)
target_numbers = np.array(eval(first_combo), dtype=float)  # 문자열을 리스트로 변환 후 NumPy 배열로 변환

print(f"6/45 로또 총 가능 조합 #: {len(set(combinations(range(1, 46), 6)))}")
print(f"이미 당첨된 로또 조합 #: {len(winner_set)}")

# 가능한 모든 조합 생성 (winner_set 제외)
valid_combinations = get_lottery_combinations(winner_set)

print(f"실제 검사 조합: {len(valid_combinations)}")

# 상위 5개 유사한 조합 찾기
top_5_similar = find_top_similar(valid_combinations, target_numbers)

end_time = time.time()

print(f"\n실행 시간: {end_time - start_time:.2f}초")

print("\n코사인 유사도가 가장 높은 5개 조합:")
for comb, similarity in top_5_similar:
    print(f"{comb}: {similarity}")

첫 번째 조합: [7, 11, 16, 21, 27, 33], 유사도: 0.9823740732197191
6/45 로또 총 가능 조합 #: 8145060
이미 당첨된 로또 조합 #: 1139
실제 검사 조합: 8143921

실행 시간: 37.66초

코사인 유사도가 가장 높은 5개 조합:
(8, 13, 19, 25, 32, 39): 0.9999852071960997
(9, 14, 21, 27, 35, 43): 0.9999740911892472
(10, 15, 22, 29, 37, 45): 0.9999726795223001
(6, 9, 13, 17, 22, 27): 0.9999688587421534
(9, 15, 21, 28, 36, 44): 0.9999656330120504
